# Stock Data Project 
Objective: Predict future stock prices (e.g., next day, week, or month) using historical data and external factors.

Type of Prediction:

Regression: Predict exact future prices.

Classification: Predict price movement direction (up/down/stable).

# Data Collection

In [1]:
import pandas as pd
import pyodbc


* Data Retreival From Alpha Vantage

API Key to Alpha Vantage (source for data): HPUL5XC5C1RFHRAQ

In [2]:
function = 'TIME_SERIES_DAILY'
symbol = 'IBM'
outputsize = 'full'
datatype = 'csv'
apikey = 'HPUL5XC5C1RFHRAQ'


url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&outputsize={outputsize}&apikey={apikey}&datatype={datatype}'
df = pd.read_csv(url)
df.sort_values(by = 'timestamp', inplace = True)
print(df)

       timestamp     open     high       low   close    volume
6345  1999-11-01   98.500   98.810   96.3700   96.75   9551800
6344  1999-11-02   96.750   96.810   93.6900   94.81  11105400
6343  1999-11-03   95.870   95.940   93.5000   94.37  10369100
6342  1999-11-04   94.440   94.440   90.0000   91.56  16697600
6341  1999-11-05   92.750   92.940   90.1900   90.25  13737600
...          ...      ...      ...       ...     ...       ...
4     2025-01-16  219.690  222.680  217.3800  222.66   3329060
3     2025-01-17  225.955  225.955  223.6400  224.79   5506837
2     2025-01-21  224.990  227.450  222.8302  224.26   3982203
1     2025-01-22  221.980  224.400  220.3500  223.26   4759490
0     2025-01-23  223.940  226.040  223.1500  226.04   3619651

[6346 rows x 6 columns]


# Data Cleaning 

Using Data Wrangler the data seems pretty clean but to ensure data integrity, I'm going to use some data cleaning methods using pandas.

We have to ensure there's no missing data, a consistent format, removal of unwanted observations, incorrect data, and removal of duplicates. We don't need to manage outliers.

* Removal of rows with missing data

In [3]:
df = df.dropna()

* Ensures that the dates are in the correct format

In [4]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

* Ensures that there is no wrong data

In [5]:
columns = list(df.columns)
columns.remove('timestamp')
columns.remove('volume')

# Ensure that the columns are of the correct data type
for column in columns:
    df[column] = pd.to_numeric(df[column], errors='coerce', downcast='float')

df['volume'] = pd.to_numeric(df['volume'], errors='coerce', downcast='integer')

#Make sure that all numbers are positive, if not make it positive
columns.append('volume')
for column in columns:
    for i in df.index:
        if df.loc[i, column] < 0:
            df.loc[i, column] = df.loc[i, column] * -1

* Removing duplicates 

In [6]:
df.drop_duplicates(inplace = True)

# Data Transforming

* Slight formating for column names

In [7]:
df.rename(columns = {'timestamp':'Date', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume':'Volume'}, inplace = True)
print(df.head())

           Date       Open       High        Low      Close    Volume
6345 1999-11-01  98.500000  98.809998  96.370003  96.750000   9551800
6344 1999-11-02  96.750000  96.809998  93.690002  94.809998  11105400
6343 1999-11-03  95.870003  95.940002  93.500000  94.370003  10369100
6342 1999-11-04  94.440002  94.440002  90.000000  91.559998  16697600
6341 1999-11-05  92.750000  92.940002  90.190002  90.250000  13737600


# Data Loading 

* Loading the data into the SQL Database provided by the cloud service Azure. This only needs to happen once.

In [8]:
'''server = 'dahomey.database.windows.net'
database = 'Stock Data'
username = 'ttshiamala'
password = 'Bear8486!?'
driver = '{ODBC Driver 18 for SQL Server}'

# Connect to Azure SQL
connection_string = f"DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}"
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()
print("Connected to Azure SQL Database!")
try:
    cursor.execute('TRUNCATE TABLE StockPrices')
    conn.commit()
    for index, row in df.iterrows():
        cursor.execute(
            """
            INSERT INTO StockPrices ([Date], [Open], [High], [Low], [Close], [Volume]) 
            VALUES (?, ?, ?, ?, ?, ?)
            """,
            row.Date, row.Open, row.High, row.Low, row.Close, row.Volume
    )
except pyodbc.IntegrityError as e:
    print(e)


conn.commit()
cursor.close()
conn.close()
print('Data inserting is a success!')'''

'server = \'dahomey.database.windows.net\'\ndatabase = \'Stock Data\'\nusername = \'ttshiamala\'\npassword = \'Bear8486!?\'\ndriver = \'{ODBC Driver 18 for SQL Server}\'\n\n# Connect to Azure SQL\nconnection_string = f"DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}"\nconn = pyodbc.connect(connection_string)\ncursor = conn.cursor()\nprint("Connected to Azure SQL Database!")\ntry:\n    cursor.execute(\'TRUNCATE TABLE StockPrices\')\n    conn.commit()\n    for index, row in df.iterrows():\n        cursor.execute(\n            """\n            INSERT INTO StockPrices ([Date], [Open], [High], [Low], [Close], [Volume]) \n            VALUES (?, ?, ?, ?, ?, ?)\n            """,\n            row.Date, row.Open, row.High, row.Low, row.Close, row.Volume\n    )\nexcept pyodbc.IntegrityError as e:\n    print(e)\n\n\nconn.commit()\ncursor.close()\nconn.close()\nprint(\'Data inserting is a success!\')'

# More Data 

* This is where I will be collecting more data for the AI to learn from as well as do some feature engineering. A lot of repeated steps for previous.

In [9]:
import requests
url = f'https://www.alphavantage.co/query?function=DIVIDENDS&symbol={symbol}&apikey={apikey}'

r = requests.get(url)
data = r.json()["data"]

df_div = pd.DataFrame(data)
print(df_div)

    ex_dividend_date declaration_date record_date payment_date amount
0         2024-11-12       2024-10-30  2024-11-12   2024-12-10   1.67
1         2024-08-09       2024-07-29  2024-08-09   2024-09-10   1.67
2         2024-05-09       2024-04-30  2024-05-10   2024-06-10   1.67
3         2024-02-08       2024-01-30  2024-02-09   2024-03-09   1.66
4         2023-11-09       2023-10-30  2023-11-10   2023-12-09   1.66
..               ...              ...         ...          ...    ...
99        2000-02-08             None        None         None   0.12
100       1999-11-08             None        None         None   0.12
101       1999-08-06             None        None         None   0.12
102       1999-05-06             None        None         None   0.24
103       1999-02-08             None        None         None   0.22

[104 rows x 5 columns]


In [10]:
df_div.drop(df_div.columns[1:4], axis = 1, inplace=True)
print(df_div.head())

  ex_dividend_date amount
0       2024-11-12   1.67
1       2024-08-09   1.67
2       2024-05-09   1.67
3       2024-02-08   1.66
4       2023-11-09   1.66


In [11]:
url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={apikey}'
r = requests.get(url)
data = r.json()["data"]
df_split = pd.DataFrame(data)

print(df_split)

  effective_date split_factor
0     2021-11-04       1.0460
1     1999-05-27       2.0000


In [12]:
df_div['ex_dividend_date'] = pd.to_datetime(df_div['ex_dividend_date'])
df_div['amount'] = pd.to_numeric(df_div['amount'], errors='coerce',downcast='float')

df_split['effective_date'] = pd.to_datetime(df_split['effective_date'])
df_split['split_factor'] = pd.to_numeric(df_split['split_factor'], errors='coerce', downcast ='float')

print(df_div.dtypes)
print(df_split.dtypes)

ex_dividend_date    datetime64[ns]
amount                     float32
dtype: object
effective_date    datetime64[ns]
split_factor             float32
dtype: object


In [13]:

df['SMA_10'] = df['Close'].rolling(window = 10).mean()
df['SMA_50'] = df['Close'].rolling(window = 50).mean()
df['SMA_100'] = df['Close'].rolling(window = 100).mean()
df['SMA_200'] = df['Close'].rolling(window = 200).mean()
print(df.tail())

        Date        Open        High         Low       Close   Volume  \
4 2025-01-16  219.690002  222.679993  217.380005  222.660004  3329060   
3 2025-01-17  225.955002  225.955002  223.639999  224.789993  5506837   
2 2025-01-21  224.990005  227.449997  222.830200  224.259995  3982203   
1 2025-01-22  221.979996  224.399994  220.350006  223.259995  4759490   
0 2025-01-23  223.940002  226.039993  223.149994  226.039993  3619651   

       SMA_10      SMA_50   SMA_100    SMA_200  
4  220.998999  221.585001  219.1539  198.84455  
3  221.483998  221.954400  219.4408  199.02410  
2  221.644998  222.288200  219.7036  199.19090  
1  221.703998  222.481400  219.9489  199.36750  
0  221.911996  222.728400  220.2247  199.55200  


In [14]:
df['EMA_10'] = df['Close'].ewm(span=10).mean()
df['EMA_50'] = df['Close'].ewm(span=50).mean()
df['EMA_100'] = df['Close'].ewm(span=100).mean()
df['EMA_200'] = df['Close'].ewm(span=2000).mean()
print(df.tail())

        Date        Open        High         Low       Close   Volume  \
4 2025-01-16  219.690002  222.679993  217.380005  222.660004  3329060   
3 2025-01-17  225.955002  225.955002  223.639999  224.789993  5506837   
2 2025-01-21  224.990005  227.449997  222.830200  224.259995  3982203   
1 2025-01-22  221.979996  224.399994  220.350006  223.259995  4759490   
0 2025-01-23  223.940002  226.039993  223.149994  226.039993  3619651   

       SMA_10      SMA_50   SMA_100    SMA_200      EMA_10      EMA_50  \
4  220.998999  221.585001  219.1539  198.84455  221.023838  221.606832   
3  221.483998  221.954400  219.4408  199.02410  221.708593  221.731662   
2  221.644998  222.288200  219.7036  199.19090  222.172484  221.830812   
1  221.703998  222.481400  219.9489  199.36750  222.370213  221.886859   
0  221.911996  222.728400  220.2247  199.55200  223.037446  222.049727   

      EMA_100     EMA_200  
4  215.850680  152.958646  
3  216.027696  153.030568  
2  216.190712  153.101887  
1  2

In [15]:
import ta

df['RSI'] = ta.momentum.rsi(df['Close'], window = 14, fillna = True)
df['MACD'] = ta.trend.macd(df['Close'], fillna = True)
df['MACD_signal'] = ta.trend.macd_signal(df['Close'], fillna = True)
df['Stochastic'] = ta.momentum.stoch(df['High'], df['Low'], df['Close'], fillna = True)
df['Bollinger_High'] = ta.volatility.bollinger_hband(df['Close'], fillna = True)
df['Bollinger_Low'] = ta.volatility.bollinger_lband(df['Close'], fillna = True)
df['ATR'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'], fillna = True)
df['OBV'] = ta.volume.on_balance_volume(df['Close'], df['Volume'], fillna = True)
df['Chaikin'] = ta.volume.chaikin_money_flow(df['High'], df['Low'], df['Close'], df['Volume'], fillna = True)
df['ADX'] = ta.trend.adx(df['High'], df['Low'], df['Close'], fillna = True)
df['SAR_Down'] = ta.trend.psar_down(df['High'], df['Low'], df['Close'], fillna = True)
df['SAR_Up'] = ta.trend.psar_up(df['High'], df['Low'], df['Close'], fillna = True)
df['MA_Crossover'] = df['SMA_10'] - df['SMA_50']
df['BB_Width'] = df['Bollinger_High'] - df['Bollinger_Low']
print(df.tail())

        Date        Open        High         Low       Close   Volume  \
4 2025-01-16  219.690002  222.679993  217.380005  222.660004  3329060   
3 2025-01-17  225.955002  225.955002  223.639999  224.789993  5506837   
2 2025-01-21  224.990005  227.449997  222.830200  224.259995  3982203   
1 2025-01-22  221.979996  224.399994  220.350006  223.259995  4759490   
0 2025-01-23  223.940002  226.039993  223.149994  226.039993  3619651   

       SMA_10      SMA_50   SMA_100    SMA_200  ...  Bollinger_High  \
4  220.998999  221.585001  219.1539  198.84455  ...      227.296598   
3  221.483998  221.954400  219.4408  199.02410  ...      226.231592   
2  221.644998  222.288200  219.7036  199.19090  ...      226.490727   
1  221.703998  222.481400  219.9489  199.36750  ...      226.410646   
0  221.911996  222.728400  220.2247  199.55200  ...      226.854383   

   Bollinger_Low       ATR        OBV   Chaikin        ADX    SAR_Down  \
4     216.753400  4.393333  362926052 -0.013994  16.206601  

In [16]:
url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=daily&apikey={apikey}&datatype=csv'
df_fed = pd.read_csv(url)

url = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey={apikey}&datatype=csv'
df_inflation = pd.read_csv(url)

url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey={apikey}&datatype=csv'
df_unemployment = pd.read_csv(url)

url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={apikey}'
r = requests.get(url)
dataAnnual = r.json()['annualEarnings']
dataQuarter = r.json()['quarterlyEarnings']
df_annual_earnings = pd.DataFrame(dataAnnual)
df_quarterly_earnings = pd.DataFrame(dataQuarter)

url = f'https://www.alphavantage.co/query?function=ALL_COMMODITIES&interval=monthly&apikey={apikey}&datatype=csv'
df_commodities = pd.read_csv(url)

print(df_fed)
print(df_inflation)
print(df_unemployment)
print(df_annual_earnings)
print(df_quarterly_earnings)
print(df_commodities)

        timestamp  value
0      2025-01-22   4.33
1      2025-01-21   4.33
2      2025-01-20   4.33
3      2025-01-19   4.33
4      2025-01-18   4.33
...           ...    ...
25769  1954-07-05   0.88
25770  1954-07-04   1.25
25771  1954-07-03   1.25
25772  1954-07-02   1.25
25773  1954-07-01   1.13

[25774 rows x 2 columns]
       timestamp    value
0     2024-12-01  315.605
1     2024-11-01  315.493
2     2024-10-01  315.664
3     2024-09-01  315.301
4     2024-08-01  314.796
...          ...      ...
1339  1913-05-01    9.700
1340  1913-04-01    9.800
1341  1913-03-01    9.800
1342  1913-02-01    9.800
1343  1913-01-01    9.800

[1344 rows x 2 columns]
      timestamp  value
0    2024-12-01    4.1
1    2024-11-01    4.2
2    2024-10-01    4.1
3    2024-09-01    4.1
4    2024-08-01    4.2
..          ...    ...
919  1948-05-01    3.5
920  1948-04-01    3.9
921  1948-03-01    4.0
922  1948-02-01    3.8
923  1948-01-01    3.4

[924 rows x 2 columns]
   fiscalDateEnding reportedEPS
0    

In [17]:
url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&apikey={apikey}&sort=LATEST&limit=1000'
r = requests.get(url)
data = r.json()['feed']
df_sentiments = pd.DataFrame(data)
print(df_sentiments)

                                                 title  \
0     Palantir Technologies Stock: Buy, Sell, or Hold?   
1    IBM Partners With e& to Launch Multi-Model AI ...   
2    IBM Stock Before Q4 Earnings: A Smart Buy or R...   
3    Apple To $280? Here Are 10 Top Analyst Forecas...   
4         This AI Stock Is Also a Great Dividend Stock   
..                                                 ...   
659  Mobile Value-added Services (VAS) Market to gr...   
660  Electronic Shift Operations Management Solutio...   
661  Data-as-a-Service (DaaS) Market size to grow b...   
662  Cyber Weapon Market to grow by USD 8.08 billio...   
663  Cyber Security Market size to grow by USD 203....   

                                                   url   time_published  \
0    https://www.fool.com/investing/2025/01/24/pala...  20250124T110000   
1    https://www.zacks.com/stock/news/2402332/ibm-p...  20250123T161100   
2    https://www.zacks.com/stock/news/2402077/ibm-s...  20250123T142600   
3  

In [18]:
df['Lag 1'] = df['Close'].shift(1)
df['7 Day Avg'] = df['Close'].rolling(window=7).mean()
df['Daily Returns'] = (df['Close'] - df['Open']) / df['Open']
df['Price to Volume Ratio'] = df['Close'] / df['Volume']
df['Day of the Week'] = df['Date'].dt.dayofweek
df['Quarter'] = df['Date'].dt.quarter
df['Daily Return'] = df['Close'].pct_change()
df['Volatility'] = df['High'] - df['Low']
df['Price Volume Interaction'] = df['Daily Return'] * df['Volume']
df['RSI * Volume'] = df['RSI'] * df['Volume']
df['MACD / Bollinger Band Width'] = df['MACD'] / df['BB_Width']

Finlight key = sk_4470114c6d50ee45e7309af37287f659d87a2b8255f771e2d6815c36864bce5a

In [19]:
dropColumns = [1,3,5,6,7,8,9]

df_sentiments.drop(df_sentiments.columns[dropColumns], axis = 1, inplace=True)

df_sentiments['date'] = pd.to_datetime(df_sentiments['time_published'], format = '%Y%m%dT%H%M%S')
df_sentiments['Date'] = df_sentiments['date'].dt.date

dropColumns = [1, 6]
df_sentiments.drop(df_sentiments.columns[dropColumns], axis = 1, inplace = True)


print(df_sentiments.head())

                                               title  \
0   Palantir Technologies Stock: Buy, Sell, or Hold?   
1  IBM Partners With e& to Launch Multi-Model AI ...   
2  IBM Stock Before Q4 Earnings: A Smart Buy or R...   
3  Apple To $280? Here Are 10 Top Analyst Forecas...   
4       This AI Stock Is Also a Great Dividend Stock   

                                             summary  overall_sentiment_score  \
0  Palantir Technologies ( NASDAQ: PLTR ) cemente...                 0.382449   
1  IBM collaborates with e& to launch a cutting-e...                 0.332855   
2  With declining earnings estimates, IBM is witn...                 0.254890   
3  Top Wall Street analysts changed their outlook...                 0.244496   
4  When you think about cutting-edge artificial i...                 0.223838   

  overall_sentiment_label                                   ticker_sentiment  \
0                 Bullish  [{'ticker': 'MSFT', 'relevance_score': '0.0589...   
1        Somewha

In [20]:
from textblob import TextBlob
rows = len(df_sentiments.iloc[:,0])
columns = ['title', 'summary', 'overall_sentiment_label']
df_TB_sentiments = df_sentiments.copy()
for column in columns:
    for row in range(0, rows):
        df_TB_sentiments.loc[row, column] = TextBlob(df_TB_sentiments.loc[row, column]).sentiment.polarity

print(df_TB_sentiments)

        title   summary  overall_sentiment_score overall_sentiment_label  \
0         0.0    0.0375                 0.382449                     0.0   
1         0.0       0.0                 0.332855                     0.0   
2    0.214286      -0.3                 0.254890                     0.0   
3         0.5  0.366667                 0.244496                     0.0   
4         0.8      -0.2                 0.223838                     0.0   
..        ...       ...                      ...                     ...   
659       0.0       0.0                 0.229583                     0.0   
660       0.0       0.0                 0.307821                     0.0   
661       0.0       0.0                 0.295652                     0.0   
662       0.0       0.0                 0.136210                     0.0   
663       0.0       0.0                 0.261836                     0.0   

                                      ticker_sentiment        Date  
0    [{'ticker': '

In [21]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

df_vader_sentiments = df_sentiments.copy()
columns = ['title', 'summary', 'overall_sentiment_label']
analyzer = SentimentIntensityAnalyzer()
for column in columns:
    df_vader_sentiments[column] = df_vader_sentiments[column].apply(lambda x: analyzer.polarity_scores(x)['compound'])
print(df_vader_sentiments.head())


    title  summary  overall_sentiment_score  overall_sentiment_label  \
0  0.0000   0.8834                 0.382449                      0.0   
1  0.5267   0.3182                 0.332855                      0.0   
2  0.2263  -0.7096                 0.254890                      0.0   
3  0.2023   0.5994                 0.244496                      0.0   
4  0.6249   0.7506                 0.223838                      0.0   

                                    ticker_sentiment        Date  
0  [{'ticker': 'MSFT', 'relevance_score': '0.0589...  2025-01-24  
1  [{'ticker': 'IDCC', 'relevance_score': '0.1478...  2025-01-23  
2  [{'ticker': 'MSFT', 'relevance_score': '0.0804...  2025-01-23  
3  [{'ticker': 'META', 'relevance_score': '0.1549...  2025-01-23  
4  [{'ticker': 'IBM', 'relevance_score': '0.45149...  2025-01-22  


In [22]:
df_TB_sentiments.drop(df_TB_sentiments.columns[[3, 4]], axis = 1, inplace = True)
df_vader_sentiments.drop(df_vader_sentiments.columns[[3, 4]], axis = 1, inplace = True)

In [23]:
import json

with open('ticker_sentiments.json', 'w') as file:
    json.dump(df_sentiments['ticker_sentiment'][0], file, indent = 4)
print('1')

1


In [24]:

filePath = r"C:\Users\ty725\OneDrive\Documents\Winter Project\ticker_sentiments.json"
df_ticker_sentiments = pd.read_json(filePath)
print(df_ticker_sentiments)

  ticker  relevance_score  ticker_sentiment_score ticker_sentiment_label
0   MSFT         0.058903                0.022053                Neutral
1    IBM         0.058903                0.022053                Neutral
2   PLTR         0.117486                0.341037       Somewhat-Bullish


I figured this out kind of, just not sure how to incorporate this into my data. I will look into this in the future but for now we'll leave it. Will also figure out finlight api later

In [25]:
SectorTicker = {'COMMUNICATION SERVICES': 'XLC',
              'CONSUMER DISCRETIONARY': 'XLY',
              'CONSUMER STAPLES': 'XLP',
              'ENERGY': 'XLE',
              'FINANCIALS': 'XLF',
              'HEALTHCARE': 'XLV',
              'INDUSTRIALS': 'XLI',
              'MATERIALS': 'XLB',
              'REAL ESTATE': 'XLRE',
              'TECHNOLOGY': 'XLK',
              'UTILITIES': 'XLU'}

url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={apikey}'
r = requests.get(url)
sectorName = r.json()['Sector']
sectorSymbol = SectorTicker[sectorName]

url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={sectorSymbol}&apikey={apikey}&datatype=csv&outputsize=full'
df_sector = pd.read_csv(url)
df_sector.sort_values(by = 'timestamp', inplace = True)
print(df_sector)



       timestamp    open    high       low   close   volume
6345  1999-11-01   42.63   42.97   42.4100   42.44   230800
6344  1999-11-02   42.44   42.94   42.1400   42.25   156300
6343  1999-11-03   42.88   43.13   42.6300   42.88   209200
6342  1999-11-04   43.44   43.47   42.9100   43.20   409100
6341  1999-11-05   44.25   44.25   43.4700   43.67   898000
...          ...     ...     ...       ...     ...      ...
4     2025-01-16  234.18  234.33  230.4500  230.50  2684261
3     2025-01-17  234.66  234.94  233.0800  234.11  3371692
2     2025-01-21  234.89  236.64  233.2601  236.06  3372076
1     2025-01-22  239.25  241.95  238.9100  241.39  5121877
0     2025-01-23  239.53  241.87  239.3000  241.82  3802235

[6346 rows x 6 columns]


# More Data Cleaning
Just to make sure

In [26]:
print(df.dtypes)


Date                           datetime64[ns]
Open                                  float32
High                                  float32
Low                                   float32
Close                                 float32
Volume                                  int32
SMA_10                                float64
SMA_50                                float64
SMA_100                               float64
SMA_200                               float64
EMA_10                                float64
EMA_50                                float64
EMA_100                               float64
EMA_200                               float64
RSI                                   float64
MACD                                  float64
MACD_signal                           float64
Stochastic                            float64
Bollinger_High                        float64
Bollinger_Low                         float64
ATR                                   float64
OBV                               

In [27]:
columns = list(df.columns)
for column in columns:
    df[column] = df[column].bfill()

In [28]:
column = df_annual_earnings.columns[1] 
df_annual_earnings[column] = pd.to_numeric(df_annual_earnings[column], downcast='float')

column = df_annual_earnings.columns[0]
df_annual_earnings[column] = pd.to_datetime(df_annual_earnings[column])
print(df_annual_earnings.dtypes)


fiscalDateEnding    datetime64[ns]
reportedEPS                float32
dtype: object


In [29]:
column = df_commodities.columns[0]
df_commodities[column] = pd.to_datetime(df_commodities[column])

column = df_commodities.columns[1] 
df_commodities[column] = pd.to_numeric(df_commodities[column], errors='coerce', downcast='float')
print(df_commodities.dtypes)
df_commodities.dropna(inplace=True)


timestamp    datetime64[ns]
value               float32
dtype: object


In [30]:
print(df_div.dtypes)

ex_dividend_date    datetime64[ns]
amount                     float32
dtype: object


In [31]:
column = df_fed.columns[0]
df_fed[column] = pd.to_datetime(df_fed[column])
print(df_fed.dtypes)

timestamp    datetime64[ns]
value               float64
dtype: object


In [32]:
column = df_inflation.columns[0]
df_inflation[column] = pd.to_datetime(df_inflation[column])
print(df_inflation.dtypes)

timestamp    datetime64[ns]
value               float64
dtype: object


In [33]:
df_quarterly_earnings.drop(columns='fiscalDateEnding', inplace=True)

In [34]:
column = df_quarterly_earnings.columns[0]
df_quarterly_earnings[column] = pd.to_datetime(df_quarterly_earnings[column])

columns = list(df_quarterly_earnings.columns)

for num in range(1, len(columns) - 1):
    df_quarterly_earnings[columns[num]] = pd.to_numeric(df_quarterly_earnings[columns[num]], downcast='float')
df_quarterly_earnings['reportTime'] = df_quarterly_earnings['reportTime'].apply(lambda x: 1 if x == 'post-market' else -1 if x == 'pre-market' else 0)
df_quarterly_earnings['reportTime'] = pd.to_numeric(df_quarterly_earnings['reportTime'], downcast='integer')
print(df_quarterly_earnings.dtypes)

reportedDate          datetime64[ns]
reportedEPS                  float32
estimatedEPS                 float32
surprise                     float32
surprisePercentage           float32
reportTime                      int8
dtype: object


In [35]:
print(df_sector.dtypes)

timestamp     object
open         float64
high         float64
low          float64
close        float64
volume         int64
dtype: object


In [36]:
print(df_split.dtypes)

effective_date    datetime64[ns]
split_factor             float32
dtype: object


In [37]:
df_TB_sentiments.drop(df_TB_sentiments.columns[[0, 1]], axis = 1, inplace=True)
print(df_TB_sentiments.dtypes)

overall_sentiment_score    float64
Date                        object
dtype: object


In [38]:
column = df_TB_sentiments.columns[1]
df_TB_sentiments[column] = pd.to_datetime(df_TB_sentiments[column])


In [39]:
column = df_unemployment.columns[0]
df_unemployment[column] = pd.to_datetime(df_unemployment[column])
print(df_unemployment.dtypes)

timestamp    datetime64[ns]
value               float64
dtype: object


In [40]:
df_vader_sentiments.drop(df_vader_sentiments.columns[0], axis = 1, inplace=True)

In [41]:
df_vader_sentiments['Date'] = pd.to_datetime(df_vader_sentiments['Date'])
print(df_vader_sentiments.dtypes)

summary                           float64
overall_sentiment_score           float64
Date                       datetime64[ns]
dtype: object


I dont think the textblob/vader experiment worked. The data looks a bit off so we're just going to stick with the overall sentiment score directly from the API.

In [42]:
df_overall_sentiments = df_sentiments.copy()
columns = [0, 1, 3, 4]
df_overall_sentiments.drop(df_overall_sentiments.columns[columns], axis = 1, inplace=True)
print(df_overall_sentiments.head())

   overall_sentiment_score        Date
0                 0.382449  2025-01-24
1                 0.332855  2025-01-23
2                 0.254890  2025-01-23
3                 0.244496  2025-01-23
4                 0.223838  2025-01-22


In [43]:
date = 'Date'
df_annual_earnings.rename(columns = {'fiscalDateEnding': date, 'reportedEPS': 'AnnualEarnings'}, errors='ignore', inplace = True)
df_commodities.rename(columns={'timestamp': date, 'value': 'Commodities_Index'}, errors='ignore', inplace=True)
df_div.rename(columns = {'ex_dividend_date': date, 'amount':'Dividend'}, errors='ignore', inplace = True)
df_fed.rename(columns = {'timestamp': date, 'value':'Fed_Rate'}, errors='ignore', inplace=True)
df_inflation.rename(columns = {'timestamp': date, 'value':'Inflation'}, errors='ignore', inplace=True)
df_quarterly_earnings.rename(columns = {'reportedDate': date, 'reportedEPS': 'QuarterEarnings', 'estimatedEPS': 'EstimatedQuarterEarnings', 'surprise': 'QuarterSurprise', 'surprisePercentage': 'QuarterSurprisePercentage', 'reportTime': 'QuarterReportTime'}, errors='ignore', inplace = True)
df_sector.rename(columns = {'timestamp': date, 'open': 'SectorOpen', 'high':'SectorHigh', 'low':'SectorLow', 'close': 'SectorClose', 'volume':'SectorVolume'}, errors='ignore', inplace = True)
df_split.rename(columns = {'effective_date': date}, errors='ignore', inplace = True)
df_unemployment.rename(columns = {'timestamp': date, 'value':'UnemploymentRate'}, errors='ignore', inplace = True)


# More Date Loading 

In [44]:
from sqlalchemy import create_engine
server = 'dahomey.database.windows.net'
database = 'Stock Data'
username = 'ttshiamala'
password = 'Bear8486!?'
driver = '{ODBC Driver 18 for SQL Server}'

# Connect to Azure SQL
connection_string = f"DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}"
conn = pyodbc.connect(connection_string)

engine = create_engine('mssql+pyodbc://', creator=lambda: conn)
df.to_sql('Stock', engine, if_exists = 'replace', index=False)

print('Data inserting is a success!')

Data inserting is a success!


In [45]:
list_df = [df_annual_earnings, df_commodities, df_div, df_fed, df_inflation, df_overall_sentiments, df_quarterly_earnings, df_sector, df_split, df_unemployment]
list_tables = ['AnnualEarnings', 'Commodities', 'Dividends', 'FederalFundsRate', 'Inflation', 'Sentiments', 'QuarterlyEarnings', 'SectorPrices', 'Split', 'Unemployment']

conn = pyodbc.connect(connection_string)
engine = create_engine('mssql+pyodbc://', creator=lambda: conn)
for i, dataFrame in enumerate(list_df):
    dataFrame.to_sql(list_tables[i], engine, if_exists = 'replace', index=False)
conn.close()

# SQL Query and Loading
This query joins all the data tables that was collected and loaded onto the database

In [51]:
with open("Query.sql", "r") as file:
    query = file.read()

conn = pyodbc.connect(connection_string)
engine = create_engine('mssql+pyodbc://', creator=lambda: conn)
finalDF = pd.read_sql(sql=query, con=engine, parse_dates=['Date'])
conn.close()

Next steps is to make sure that the data makes sense. For example, with unemployment, not all values were added because of left join so I may have to revisit how I joined these tables. Need to account for days that the stock wasn't trading (commodities as well). Also have to double check sentiments. Need to also consider the case that there were multiple stories in one day (average). After that I have to clean the dataset once more and handle missing values and things of that nature. Actually now that I'm taking a closer look at it, there are some spurious tables. It may be because of the overall sentiments for example with multiple dates.

I fixed it the dataset is almost perfect. Just need to clean it one more time then it is ready for machine learning.